In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from math import sin, cos, sqrt, atan2, radians
import re
import warnings
warnings.filterwarnings('ignore')
print('Libraries imported!')

Libraries imported!


In [2]:
calgary_df = pd.read_csv('data/calgary_listings.csv')
calgary_df.head()

,price,postal_code,address,mls,bed,full_bath,half_bath,property_area,property_type,latitude,longitude,year_built,years_old,distance_downtown,monthly_payments,lot_size,basement,garage
0,649900,T3G 6G6,14 Norford Common NW,A1046940,2,2,1.0,1560,townhouse,51.080529,-114.145,2018.0,2.0,6.35,2810.25,na,0.0,1.0
1,714800,T3H 4Y3,171 DISCOVERY RIDGE BLVD SW,A1046396,4,3,1.0,2242,house,51.015494,-114.214,2011.0,9.0,10.33,3090.88,6738,1.0,1.0
2,649900,T2X 0K3,20 SILVERADO SKIES MANOR SW,A1043396,5,3,NaN,2360,house,50.888762,-114.083,1958.0,62.0,17.38,2810.25,4467,1.0,1.0
3,349500,T2E 3S7,"202, 707 4 Street NE",A1046769,2,2,NaN,938,condo,51.057467,-114.053,2010.0,10.0,2.02,1511.28,NaN,0.0,1.0
4,400000,T2E 7Y1,213 - 46 9 ST NE,A1046980,2,2,NaN,891,condo,51.050504,-114.039,2016.0,4.0,2.52,1729.65,NaN,0.0,1.0


In [3]:
post_code_list = []

for i in calgary_df["postal_code"]:
    post_code_list.append(i)

In [4]:
scores_walk = []
scores_bike = []
scores_transit = []

for i in post_code_list:

    time.sleep(5)
    
    try:
        postal_code = i.replace(" ", "%20")
        url_score = "https://www.walkscore.com/score/" + str(postal_code)
        time.sleep(5)

        # Parse HTML with Beautiful Soup
        response = requests.get(url_score)
        code_soup = BeautifulSoup(response.text, 'html.parser')

        if 'pp.walk.sc/badge/walk/score' in str(code_soup):
            ws = str(code_soup).split('pp.walk.sc/badge/walk/score/')[1][:2].replace('.','')
            scores_walk.append(ws)
        else:
            ws = 0
            scores_walk.append(ws)
        if 'pp.walk.sc/badge/bike/score' in str(code_soup):
            bs = str(code_soup).split('pp.walk.sc/badge/bike/score/')[1][:2].replace('.','')
            scores_bike.append(bs)
        else:
            bs = 0
            scores_bike.append(bs)
        if 'pp.walk.sc/badge/transit/score' in str(code_soup):
            ts = str(code_soup).split('pp.walk.sc/badge/transit/score/')[1][:2].replace('.','')
            scores_transit.append(ts)
        else:
            ts = 0
            scores_transit.append(ts)
    except:
        ws = 0
        scores_walk.append(ws)
        bs = 0
        scores_bike.append(bs)
        ts = 0
        scores_transit.append(ts)

In [5]:
score_df_trans = {'postal_code':post_code_list, 
                  'walk_score':scores_walk, 
                  'bike_score':scores_bike, 
                  'transit_score':scores_transit}
score_df_dup = pd.DataFrame(score_df_trans)
score_df = score_df_dup.drop_duplicates()
score_df.head()

,postal_code,walk_score,bike_score,transit_score
0,T3G 6G6,35,70,35
1,T3H 4Y3,5,55,24
2,T2X 0K3,16,34,41
3,T2E 3S7,80,79,65
4,T2E 7Y1,24,54,34


In [6]:
score_df.to_csv('data/score_df.csv', index=False)